# Create batch

## Set correct directory

In [1]:
cd ..

/bptk-py


## Import libraries

In [5]:
import pandas as pd
import csv
import json
import time
import datetime
import pickle
from elasticsearch import Elasticsearch
import os.path
from src.config.conf import width,height
from src.setup import setup_model
from ipywidgets import widgets
from ipywidgets import IntSlider

## Obtain Number Of Taxis

In [10]:
with open('scenarios/abm.json') as scenario_file:
    abm = json.load(scenario_file)

num_cars=abm["CARMODEL"]["scenarios"]["scenario"]["agents"][1]["count"]
num_cars

3

## Calculate Batch Results

In [ ]:
es = Elasticsearch([{'host': 'es_node1', 'port': 9200}])


car_list = [str(i)+"_car" for i in range(1,num_cars+1)]

keys=['id','time_start','time_end','revenue','cost','profit']

data={"time":0}

for car_id, car in enumerate(car_list,1):
    data[str(car_id)+"_car"]={
        "time":0,
        "timestamp": datetime.datetime.now(), 
        "id":car_id,
        "sum_revenue":0,
        "sum_cost":0,
        "profit":0,
        "avg_sum_revenue":0,
        "avg_sum_cost":0,
        "avg_profit":0
    }
    
sleep_time=60
time_delta=1440
last_batch_processing_time=0

while True:
    
    
    # read the time
    time_file_object = open("csv/sim_time.pickle",'rb')  
    current_time = pickle.load(time_file_object)

    if current_time>=last_batch_processing_time+time_delta:
        
        data["time"]=last_batch_processing_time+time_delta
    
        for car_id, car in enumerate(car_list,1):
            time_from=last_batch_processing_time+1
            time_to=last_batch_processing_time+time_delta+1
               
            profit_result=es.search(index=car,body={"size": 0,
                "aggs" : {
                    "sum_profit" : {
                        "date_range": {
                            "field": "time",
                            "ranges": [
                                { "from": time_from,
                                  "to": time_to } 
                            ]
                        },
                        "aggs": {
                            "sum_cost": {
                              "sum": {
                                "field": "cost"
                              }
                            },
                            "sum_revenue":{
                              "sum":{
                                "field":"revenue"
                              }
                            },
                            "profit": {
                                "bucket_script": {
                                    "buckets_path": {
                                      "sumRev": "sum_revenue",
                                      "sumCost": "sum_cost"
                                    },
                                    "script": "params.sumRev - params.sumCost"
                                }
                            }
                        }
                    }
                }})

            for result in ["sum_revenue","sum_cost","profit"]:
                data[car]["time"] = data["time"]
                data[car]["timestamp"] = datetime.datetime.now() 
                data[car]["id"] = car_id
                if result in profit_result["aggregations"]["sum_profit"]["buckets"][0].keys():
                    data[car][result] += (profit_result['aggregations']['sum_profit']['buckets'][0][result]['value'])
                else:
                    data[car][result] += 0
                data[car]["avg_"+result] = data[car][result]/data[car]["time"]
            
            # send the data to elasticsearch
            es.index(index=car+"_batch", body=data[car])
                  
        
        last_batch_processing_time+=time_delta
    else:        
        time.sleep(sleep_time)